# Previous Lesson Recap
<a href='#the_destination'><b>Jump to the current lesson<b></a>

## 1. Exploratory Data Analysis
We have already about exploratory data analysis in the [Kaggle Challenge - Exploratory Data Analysis](https://www.educative.io/collection/page/4747639511842816/6260527028240384/5834129520197632) lesson.

#### Importing Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

#### Reading the CSV file (assume that the file is stored in the data folder)

In [ ]:
housing = pd.read_csv('./data/train.csv')


### Understanding the Data Structure

Get the columns in the pandas data frame using the `.columns` function

In [ ]:
# columns
housing.columns

Get the shape of the data using the `.shape` function

In [ ]:
# Get the shape of data
housing.shape

The "shape" of the dataset shows that it has 1460 rows/instances, with data from 80 attributes. 
Out of the 80 attributes, one is the target (SalePrice) that the model should predict. 
Hence, there are 79 attributes that may be used for feature selection/engineering.

Get the first five rows of the data set using the `.head()` function

In [ ]:
housing.head()

Get the description about the data using the `.info()` method

In [ ]:
housing = housing.drop("Id", axis=1)
housing.info()

Get the list of numerical attributes by excluding the datatype of object using the `exclude=['object']`

In [ ]:
# List of numerical attributes
housing.select_dtypes(exclude=['object']).columns

Get the length after excluding the columns of object type using the `len` function

In [ ]:
len(housing.select_dtypes(exclude=['object']).columns)

Get the data summary with upto 2 decimals and call `transpose()` for a better view of the results

In [ ]:
housing.select_dtypes(exclude=['object']).describe().round(decimals=2).transpose()

Get the categorical attributes by including datatype of type object using `include = ['object']`

In [ ]:
# get categorical attribute
housing.select_dtypes(include=['object']).columns

There are 43 categorical columns with the following characteristics:

Get the categorical attributes using `include=['object']`

In [ ]:
housing.select_dtypes(include=['object']).describe().transpose()

### Exploring Numerical Columns

Let’s have a detailed look at our target variable, SalePrice.

In [ ]:
# Statistics summary
housing['SalePrice'].describe()

Skew of target column
It appears to be good practice to minimise the skew of the dataset. The reason often given is that skewed data adversely affects the prediction accuracy of regression models. 
Note: While important for linear regression, correcting skew is not necessary for Decisions Trees and Random Forests. 

(Not a needed step for target var, while we can scale the other numerical features)

Get the distribution plot using the seaborn `.displot` function

In [ ]:
# Get the distribution plot
sns.displot(housing['SalePrice']);

Applying a log transformation means to simply take the log of the skewed variable to improve the fit by altering the scale and making the variable more “normally” distributed.

In [ ]:
# Take the log to make the distribution more normal
sns.displot(np.log(housing['SalePrice']))
plt.title('Distribution of Log-transformed SalePrice')
plt.xlabel('log(SalePrice)')
plt.show()

We can clearly see that the log-transformed variable is more “normally” distributed — we have managed to reduce the skew.

We can plot the distributions of all the numerical variables by calling the distplot() method in a for loop, like so:

In [ ]:
## What about the distribution of all the other numerical variables?

num_attributes = housing.select_dtypes(exclude='object').drop(['SalePrice'], axis=1).copy()

print(len(num_attributes.columns))

fig = plt.figure(figsize=(12,18))
for i in range(len(num_attributes.columns)):
    fig.add_subplot(9,4,i+1)
    sns.distplot(num_attributes.iloc[:,i].dropna(), hist = False, rug = True)
    plt.xlabel(num_attributes.columns[i])

plt.tight_layout()
plt.show()

We can clearly see how skewed LotArea is — it is in dire need of some polishing before it can be used for learning.

Since box plots give us a good overview of our data. From the distribution of observations w.r.t. the upper and lower quartiles, we can spot outliers. Let’s see this in action with the boxplot() method and a for loop to plot all the attributes in one go:

In [ ]:
fig = plt.figure(figsize=(10, 15))

for i in range(len(num_attributes.columns)):
    fig.add_subplot(9, 4, i+1)
    sns.boxplot(y=num_attributes.iloc[:,i])

plt.tight_layout()
plt.show()

In [ ]:
# return value at the given quantile over requested axis.
high_quant = housing.quantile(.999)
high_quant['LotArea']

We are going to look at the prices. The radius of each circle represents GrLivArea (option s), and the color represents the price (option c). We will use a predefined color map (option cmap) called jet, which ranges from blue (low values) to red (high prices).

In [ ]:
housing.plot(kind="scatter", x="OverallQual", y="YearBuilt",  s=housing["GrLivArea"], label="GrLivArea", alpha=0.3, 
             figsize=(10,7), c="SalePrice", cmap=plt.get_cmap("jet"), colorbar=True
)
plt.legend()

### Correlations Among Numerical Attributes

We can use the `corr()` method to very easily get the correlations and then visualize them using the `heatmap()` method – Python does feel like magic often, isn’t it?!

In [ ]:
# Correlation of numerical attributes
corr = housing.corr()

# Using mask to get triangular correlation matrix
f, ax = plt.subplots(figsize=(15, 15))
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

sns.heatmap(corr, mask=mask, cmap=sns.diverging_palette(220, 10, as_cmap=True), square=True, ax=ax, vmin = -1.0, vmax = 1.0, linewidths=.5)

With reference to the target SalePrice, the top correlated attributes are:

In [ ]:
corr['SalePrice'].sort_values(ascending=False)

From these values, we can see that OverallQual and GrLivArea have the most impact on price, while attributes like PoolArea and MoSold are not related to it.

A rough joint distribution plot for each pair of variables can be done by using `pairplot()`from sns.

In [ ]:
col = ['SalePrice', 'OverallQual', 'GrLivArea', 'YearBuilt']
sns.pairplot(housing[col])

From the pairplots, we can clearly see how with an increase in GrLivArea the price increases as well. Do play around with other attributes as well.

In [ ]:
# Only important correlations and not auto-correlations
threshold = 0.5
important_corrs = (corr[abs(corr) > threshold][corr != 1.0]) \
    .unstack().dropna().to_dict()

unique_important_corrs = pd.DataFrame(
    list(set([(tuple(sorted(key)), important_corrs[key]) \
    for key in important_corrs])), columns=['attribute pair', 'correlation'])

# sorted by absolute value
unique_important_corrs = unique_important_corrs.iloc[
    abs(unique_important_corrs['correlation']).argsort()[::-1]]

In [ ]:
unique_important_corrs

Looking at the graphs for the highest corr var

Get the bar plot

In [ ]:
sns.barplot(housing.OverallQual, housing.SalePrice)

Get the box plot

In [ ]:
#boxplot
plt.figure(figsize=(18, 8))
sns.boxplot(x=housing.OverallQual, y=housing.SalePrice)

Let’s look at the age of the house

In [ ]:
var = 'YearBuilt'
data = pd.concat([housing['SalePrice'], housing[var]], axis=1)
f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);
plt.xticks(rotation=90);

### Exploring Categorical Variables

Let’s just print again the names of the categorical columns first and then handpick some of the interesting ones for visual analysis.

In [ ]:
cat_columns = housing.select_dtypes(include='object').columns
print(cat_columns)

Let's look at the impact of KitchQual on price:

In [ ]:
var = housing['KitchenQual']
f, ax = plt.subplots(figsize=(10,6))
sns.boxplot(y=housing.SalePrice, x=var)
plt.show()

We can now see that Ex seems to be the more expensive option while Fa brings the prices down.

Let's look at the style of the house

In [ ]:
f, ax = plt.subplots(figsize=(12,8))
sns.boxplot(y=housing.SalePrice, x=housing.HouseStyle)
plt.xticks(rotation=40)
plt.show()

We can see that 2Story houses have the highest variability in prices and they also tend to be more expensive, while 1.5Unf are the cheapest option.

Get the frequency for each of these types, we can use the countplot() method from sns like so:

In [ ]:
# Count of categories within HouseStyle attribute
fig = plt.figure(figsize=(12, 4))
sns.countplot(x='HouseStyle', data=housing)
plt.xticks(rotation=90)
plt.ylabel('Frequency')
plt.show()

Get exact count for each category, we can get that very easily from the DataFrame directly:

In [ ]:
housing["HouseStyle"].value_counts()

We are also curious to see if the style of the houses has changed over the years, so let’s plot the two variables against each other.

In [ ]:
plt.scatter(housing['YearBuilt'],housing['HouseStyle'])

## 2. Data Preprocessing - Prepare the Data for Machine Learning Algorithms
We have already learned about data preprocessing in the [Kaggle Challenge - Data Preprocessing
](https://www.educative.io/collection/page/4747639511842816/6260527028240384/5263608478957568) lesson.


<a href='#the_destination'><b>Jump to the current lesson<b></a>
 
Get a sorted count of the missing values for all the attributes.

In [ ]:
housing.isnull().sum().sort_values(ascending=False)

From the results above we can assume that PoolQC to Bsmt attributes are missing for the houses that do not have these facilities (houses without pools, basements, garage etc.). Therefore, the missing values could be filled in with “None”. MasVnrType and MasVnrArea both have 8 missing values, likely houses without masonry veneer.

### Deal With Missing Values
We are going to apply different approaches to fix our missing values, so that we can various approaches in action:

* Replace values for categorical attributes with None.
* Compute the median LotFrontage for all the houses in the same neighborhood, instead of the plain median for the entire column, and use that to impute on a neighborhood by neighborhood basis.
* Replace missing values for most of the numerical columns with zero and one with the mode.
* Drop one non-interesting column, Utilities.

In [ ]:
# Imputing Missing Values

housing_processed = housing

# Categorical columns:
cat_cols_fill_none = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu',
                     'GarageCond', 'GarageQual', 'GarageFinish', 'GarageType',
                     'BsmtFinType2', 'BsmtExposure', 'BsmtFinType1', 'BsmtQual', 'BsmtCond',
                     'MasVnrType']

# Replace missing values for categorical columns with None
for cat in cat_cols_fill_none:
    housing_processed[cat] = housing_processed[cat].fillna("None")
    
# Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
housing_processed['LotFrontage'] = housing_processed.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))    

# GarageYrBlt, GarageArea and GarageCars these are numerical columns, replace with zero
for col in ['GarageYrBlt', 'GarageArea', 'GarageCars']:
    housing_processed[col] = housing_processed[col].fillna(int(0))
    
#MasVnrArea : replace with zero
housing_processed['MasVnrArea'] = housing_processed['MasVnrArea'].fillna(int(0))

#Use the mode value 
housing_processed['Electrical'] = housing_processed['Electrical'].fillna(housing_processed['Electrical']).mode()[0]

#There is no need of Utilities so let's just drop this column
housing_processed = housing_processed.drop(['Utilities'], axis=1)

In [ ]:
# Get the count again to verify that we do not have any more missing values
housing_processed.isnull().apply(sum).max()

### Deal With Outliers
Invoking the `quantile()` method on the DataFrame and then filtering based on the knowledge of the quantiles for each attribute, like so:

In [ ]:
num_attributes = housing_processed.select_dtypes(exclude='object')

high_quant = housing_processed.quantile(.999)

for i in num_attributes.columns:
    housing_processed = housing_processed.drop(housing_processed[i][housing_processed[i]>high_quant[i]].index)

housing_processed.info()

### Deal With Categorical Attributes

we can drop GarageArea because it is highly correlated with GarageCars and the reason for preferring GarageCars is because it is more correlated with price than area. 

In [ ]:
#### Remove highly correlated features
# Remove attributes that were identified for excluding when viewing scatter plots & corr values
attributes_drop = ['MiscVal', 'MoSold', 'YrSold', 'BsmtFinSF2','BsmtHalfBath','MSSubClass',
                   'GarageArea', 'GarageYrBlt', '3SsnPorch']

housing_processed = housing_processed.drop(attributes_drop, axis=1)

### Handle Text and Categorical Data
Let's convert all the categories from text to numbers.
A common approach to deal with textual data is to create one binary attribute for each category of the feature: for example, for type of houses, we would have one attribute equal to 1 when the category is 1Story (and 0 otherwise), another attribute equal to 1 when the category is 2Story (and 0 otherwise), and so on. This is called one-hot encoding, because only one attribute will be equal to 1 (hot), while the others will be 0 (cold). The new attributes are also known as dummy attributes.

In [ ]:
#### Transforming Cat variables
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_processed_1hot = cat_encoder.fit_transform(housing_processed)
housing_processed_1hot

## 3. Transformation Pipelines

We have already learned Data Preprocessing in [Kaggle Challenge - Data Transformation](https://www.educative.io/collection/page/4747639511842816/6260527028240384/5292909500825600) lesson.


<a href='#the_destination'><b>Jump to the current lesson<b></a>

Let's look at a simple example pipeline to impute and scale numerical attributes.

In [ ]:
# Import modules
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Separate features and target variable
housing_X = housing_processed.drop("SalePrice", axis=1)
housing_y = housing_processed["SalePrice"].copy()

# Get the list of names for numerical and categorical attributes separately
num_attributes = housing_X.select_dtypes(exclude='object')
cat_attributes = housing_X.select_dtypes(include='object')

num_attribs = list(num_attributes)
cat_attribs = list(cat_attributes)

# Numerical Pipeline to impute any missing values with the median and scale attributes
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])


Note that we have separated the SalePrice attribute into a separate variable, because for creating the machine learning model, we need to separate all the features, housing_X, from the target variable, housing_y.

It is more convenient and clean to have a single transformer handle all columns, applying the appropriate transformations to each column. Scikit-Learn comes to the rescue again by providing the `ColumnTransformer` for the very purpose. Let’s use it to apply all the transformations to our data and create a complete pipeline.

In [ ]:
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])


# Description before applying transforms
print(housing_y.describe())

# Apply log-transform to SalePrice
housing_y_prepared  = np.log(housing_y)

# Run the transformation pipeline on all the other attributes
housing_X_prepared = full_pipeline.fit_transform(housing_X)

# Description before applying transforms
print(housing_y_prepared.describe())

housing_X_prepared

## 4. Machine Learning Models 
We have already learned about data preprocessing in the [Kaggle Challenge - Machine Learning Models](https://www.educative.io/collection/page/4747639511842816/6260527028240384/5060702475649024) lesson.


<a href='#the_destination'><b>Jump to the current lesson<b></a>

Let's apply machine learning techniques to our transformed data.

### Select and Train Models 
We'll split the transformed data set into training and test set

In [ ]:
# Split data into train and test formate
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(housing_X_prepared, housing_y_prepared, test_size=0.2, random_state=7)

Applying **Linear Regression** Algorithm

In [ ]:
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
import xgboost

# Invert the log-transformed value
def inv_y(transformed_y):
    return np.exp(transformed_y)

# Series to collect RMSE for the different algorithms: "algorithm name + rmse"
rmse_compare = pd.Series()
rmse_compare.index.name = 'Model'

# Series to collect accuracy scores for the different algorithms: "algorithm name + score"
scores_compare = pd.Series()
scores_compare.index.name = 'Model'

# Model 1: Linear Regression =================================================
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

linear_val_predictions = linear_model.predict(X_test)
linear_val_rmse = mean_squared_error(inv_y(linear_val_predictions), inv_y(y_test))
linear_val_rmse = np.sqrt(linear_val_rmse)
rmse_compare['LinearRegression'] = linear_val_rmse

lr_score = linear_model.score(X_test, y_test)*100
scores_compare['LinearRegression'] = lr_score

Applying **Decision Trees** Algorithm

In [ ]:
# Model 2: Decision Trees. Define the model. =============================
dtree_model = DecisionTreeRegressor(random_state=5)
dtree_model.fit(X_train, y_train)

dtree_val_predictions = dtree_model.predict(X_test)
dtree_val_rmse = mean_squared_error(inv_y(dtree_val_predictions), inv_y(y_test))
dtree_val_rmse = np.sqrt(dtree_val_rmse)
rmse_compare['DecisionTree'] = dtree_val_rmse

dtree_score = dtree_model.score(X_test, y_test)*100
scores_compare['DecisionTree'] = dtree_score

Applying **Random Forest** ALgorithm

In [ ]:
# Model 3: Random Forest. Define the model. =============================
rf_model = RandomForestRegressor(random_state=5)
rf_model.fit(X_train, y_train)

rf_val_predictions = rf_model.predict(X_test)
rf_val_rmse = mean_squared_error(inv_y(rf_val_predictions), inv_y(y_test))
rf_val_rmse = np.sqrt(rf_val_rmse)
rmse_compare['RandomForest'] = rf_val_rmse

rf_score = rf_model.score(X_test, y_test)*100
scores_compare['RandomForest'] = rf_score

Applying **Gradiest Boosting** Algorithm

In [ ]:
# Model 4: Gradient Boosting Regression ==========================================
gbr_model = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, 
                                      max_depth=4, random_state=5)
gbr_model.fit(X_train, y_train)

gbr_val_predictions = gbr_model.predict(X_test)
gbr_val_rmse = mean_squared_error(inv_y(gbr_val_predictions), inv_y(y_test))
gbr_val_rmse = np.sqrt(gbr_val_rmse)
rmse_compare['GradientBoosting'] = gbr_val_rmse

gbr_score = gbr_model.score(X_test, y_test)*100
scores_compare['GradientBoosting'] = gbr_score

Calculating **Root Mean Square Error**

In [ ]:
print('RMSE values for different algorithms:')
rmse_compare.sort_values(ascending=True).round()

Calculating **Accuracy** by applying different Algorithms

In [ ]:
print('Accuracy scores for different algorithms:')
scores_compare.sort_values(ascending = False).round(3)

The simplest model, Linear Regression, seems to be performing the best, with predicted prices that are off by about 24K. This might or might not be an acceptable amount of deviation – depends on the desired level of accuracy or the metric we are trying to optimize based on our business objective.

#### Cross validation

Let’s perform a K-fold cross-validation on our best model: the cross-validation function randomly splits the training set into K distinct subsets (folds), then it trains and evaluates the model K times, picking a different fold for evaluation every time and training on the other 9 folds. The result is an array containing the K evaluation scores:

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(linear_model, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)
linear_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
display_scores(linear_rmse_scores)

In [ ]:
scores = cross_val_score(rf_model, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)
rf_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
display_scores(rf_rmse_scores)

From the results, we can notice that cross-validation gives us the mean and standard deviation for the scores as well. 

<a id='the_destination'><h2>Current Lesson</a>

## 5. Fine-Tune Your Model

This is a model that has many input hyperparameters that can be tweaked for improving performance. For example, we could have a forest with 100 or 1000 trees, or we could use 10 or 50 features during random selection. What are the best values for these hyperparameters to pass as input to the model for training?

We can use Scikit-learn’s `GridSearchCV` to tell  which hyperparameters we would like to explore and which values to try out, and it will evaluate all the possible combinations of hyperparameter values, using cross-validation.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [10, 50, 100, 150], 'max_features': [10, 20, 30, 40, 50, 100, 150]},
    {'bootstrap': [False], 'n_estimators': [10, 50, 100, 150], 'max_features': [10, 20, 30, 40, 50, 100, 150]},
  ]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(X_train, y_train)

We can use best_params_ to visualize the best values for the passed hyperparameters, and best_estimator_ to get the fine-tuned model:

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

Now that we know the optimal values for the hyperparameters (‘bootstrap’: False, ‘max_features’: 50, ‘n_estimators’: 150), let’s plug them in and see if our Random Forest model has improved compared to the vanilla Random Forest model that we trained earlier when we trained multiple models at once:

In [ ]:
rf_model_final = RandomForestRegressor(bootstrap=False,max_features=50, n_estimators=150, random_state=5)

rf_model_final.fit(X_train, y_train)
rf_final_val_predictions = rf_model_final.predict(X_test)
rf_final_val_rmse = mean_squared_error(inv_y(rf_final_val_predictions), inv_y(y_test))
np.sqrt(rf_final_val_rmse)

In [ ]:
rf_model_final.score(X_test, y_test)*100

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_

Let’s display these importance scores next to their corresponding attribute names:

In [ ]:
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)